In [1]:
!pip install torch==1.8.2+cu111 torchtext==0.9.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install -r requirements.txt

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html


ERROR: Could not find a version that satisfies the requirement torch==1.8.2+cu111 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0)
ERROR: No matching distribution found for torch==1.8.2+cu111
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [2]:
pip install ollama

Note: you may need to restart the kernel to use updated packages.


In [3]:
from ollama import Client

client = Client()
print(client.list())

models=[Model(model='deepseek-r1:latest', modified_at=datetime.datetime(2025, 2, 1, 14, 4, 1, 798831, tzinfo=TzInfo(+01:00)), digest='0a8c266910232fd3291e71e5ba1e058cc5af9d411192cf88b6d30e92b6e73163', size=4683075271, details=ModelDetails(parent_model='', format='gguf', family='qwen2', families=['qwen2'], parameter_size='7.6B', quantization_level='Q4_K_M')), Model(model='llama3.2:latest', modified_at=datetime.datetime(2025, 2, 1, 11, 11, 31, 919697, tzinfo=TzInfo(+01:00)), digest='a80c4f17acd55265feec403c7aef86be0c25983ab279d83f3bcd3abbcb5b8b72', size=2019393189, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='3.2B', quantization_level='Q4_K_M'))]


In [10]:
import ollama
import re

def extract_quotes(document: str, question: str, model: str = "deepseek-r1:latest"):
    """
    Extract relevant quotes from the document based on the given question.
    
    Parameters:
        - document (str): The input document to analyze.
        - question (str): The question for which we need to find relevant quotes.
        - model (str): The Ollama model to use (default: 'deepseek').
    
    Returns:
        - str: Extracted quotes formatted inside <quotes> tags or a message if none are found.
    """
    prompt = f"""
    You are a helpful assistant. Your task is to extract quotes from the following document that are relevant to the given question.
    
    Document:
    ####
    {document}
    ####
    
    Question: {question}
    
    Please output the relevant quotes inside <quotes></quotes> tags. If no quotes are relevant, respond with "No relevant quotes found!".
    """

    response = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}])
    
    # Extract text response
    content = response.get("message", {}).get("content", "")

    # Extract quotes using regex
    match = re.search(r"<quotes>(.*?)</quotes>", content, re.DOTALL)
    
    if match:
        return f"<quotes>{match.group(1).strip()}</quotes>"
    else:
        return "No relevant quotes found!"

def compose_answer(document: str, quotes: str, question: str, model: str = "deepseek-r1:latest"):
    """
    Compose an answer to the question using the extracted quotes and the original document.
    
    Parameters:
        - document (str): The original document.
        - quotes (str): The relevant quotes extracted from the document.
        - question (str): The question to answer.
        - model (str): The Ollama model to use (default: 'deepseek').
    
    Returns:
        - str: A composed answer to the question.
    """
    prompt = f"""
    Given a set of relevant quotes (delimited by <quotes></quotes>) extracted from a document and the original document (delimited by ####), please compose an answer to the question. Ensure that the answer is accurate, has a friendly tone, and sounds helpful.
    
    ####
    {document}
    ####
    
    <quotes>
    {quotes}
    </quotes>
    
    Question: {question}
    """

    response = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}])
    
    # Extract text response
    content = response.get("message", {}).get("content", "")

    return content

# Example Usage
question_text = "Can you give advice on what to do in case of domestic violence?"

# Load JSON file manually
with open('./data_cleaned.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Find the maximum length of arrays
max_length = max(len(arr) for arr in data.values())

# Pad shorter arrays with None
for key in data:
    data[key] = data[key] + [None] * (max_length - len(data[key]))

# Create DataFrame
document_text = pd.DataFrame(data)

# Step 1: Extract relevant quotes
quotes = extract_quotes(document_text.to_string(), question_text)

# Step 2: Compose an answer if quotes are found
if quotes != "No relevant quotes found!":
    answer = compose_answer(document_text.to_string(), quotes, question_text)
    print("Composed Answer:")
    print(answer)
else:
    print("No relevant quotes found. Unable to compose an answer.")

Composed Answer:
<think>
Okay, I need to figure out how to respond to the user's question about advice for dealing with domestic violence. First, I'll look at the provided data which includes information from various sources like phone numbers, websites, opening hours, and specific help lines.

I see that there are multiple entries related to 'Sexual Exploitation' but more importantly, two entries mention 'Domestic Violence'. Wait, actually looking back, the user's query is about domestic violence, so I should focus on that. 

Looking through the data, I notice entries with Unnamed 83 and 87 both seem relevant. Let me check their details.

Entry 83 has a phone number: Phone: 0800-0188 and mentions they provide help via chat at www.chatmetcsg.nl. They also state that calls can be made 24/7, which is great for immediate support.

Then there's entry 87, which is 'Cetrum Seksueel Geweld'. It provides phone support 24/7 and mentions that they help through chat with specific hours Monday-Fri

In [12]:
import ollama
import re
import json
import pandas as pd

def extract_quotes(document: str, question: str, model: str = "deepseek-r1:latest"):
    """
    Extract relevant quotes from the document based on the given question.
    """
    prompt = f"""
    You are a helpful assistant. Your task is to extract quotes from the following document that are relevant to the given question.

    Document:
    ####
    {document}
    ####

    Question: {question}

    Please output the relevant quotes inside <quotes></quotes> tags. If no quotes are relevant, respond with "No relevant quotes found!".
    """

    response = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}])
    
    # Extract text response
    content = response.get("message", {}).get("content", "")

    # Extract quotes using regex
    match = re.search(r"<quotes>(.*?)</quotes>", content, re.DOTALL)
    
    if match:
        return f"<quotes>{match.group(1).strip()}</quotes>"
    else:
        return "No relevant quotes found!"

def compose_answer(document: str, quotes: str, question: str, source: str, last_updated: str, model: str = "deepseek-r1:latest"):
    """
    Compose an answer to the question using the extracted quotes and the original document.
    """
    prompt = f"""
    Given a set of relevant quotes (delimited by <quotes></quotes>) extracted from a document and the original document (delimited by ####), please compose an answer to the question. Ensure that the answer is accurate, has a friendly tone, and sounds helpful.

    The response should include:
    1. A direct answer to the question using the extracted quotes.
    2. A note specifying the source and last updated date.
    3. A brief summary of the key points at the end.

    Document Source: {source}
    Last Updated: {last_updated}

    Document:
    ####
    {document}
    ####

    Relevant Quotes:
    <quotes>
    {quotes}
    </quotes>

    Question: {question}

    Format the response as follows:
    - Start with a friendly and clear answer to the question.
    - Include the source and last updated date in a separate paragraph at the end.
    - End with a **Summary:** section highlighting the key points in 2-3 sentences.
    """

    response = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}])
    
    # Extract text response
    content = response.get("message", {}).get("content", "")

    return content


# Example Usage
question_text = "Can you give advice on what to do in case of domestic violence?"

# Load JSON file manually
with open('./data_cleaned.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Find the maximum length of arrays
max_length = max(len(arr) for arr in data.values())

# Pad shorter arrays with None
for key in data:
    data[key] = data[key] + [None] * (max_length - len(data[key]))

# Create DataFrame
document_df = pd.DataFrame(data)

# Extract metadata correctly
source = document_df["Link to Website #WEBURLS"].iloc[0] if "Link to Website #WEBURLS" in document_df.columns else "Unknown Source"
last_updated = document_df["Added on (date)"].iloc[0] if "Added on (date)" in document_df.columns else "Unknown Date"

# Step 1: Extract relevant quotes
quotes = extract_quotes(document_df.to_string(), question_text)

# Step 2: Compose an answer if quotes are found
if quotes != "No relevant quotes found!":
    answer = compose_answer(document_df.to_string(), quotes, question_text, source, last_updated)
    print("Composed Answer:")
    print(answer)
else:
    print("No relevant quotes found. Unable to compose an answer.")

Composed Answer:
<think>
Okay, so I need to figure out how to respond to this user's query. They're asking about advice on what to do in case of domestic violence. Hmm, let me think.

First, I should recognize that domestic violence is a serious issue and the user probably needs reliable information. Since they mentioned Centrum Seksueel Geweld earlier, maybe they found that resource useful before. So, I should build on that.

The user provided some quotes from Centrum Seksueel Geweld, which say they provide guidance on handling domestic violence with an employer. That seems important to include because it shows the organization's role in this situation.

I need to structure the response clearly. The user wants a friendly and clear answer, followed by source info, and then a summary. So I'll start with a welcoming statement about knowing that domestic violence is serious and that Centrum can help.

Next, outline the steps: reporting to authorities, contacting an employer or HR, seeking